In [1]:
# to highlight part of a pdf file, use the following command:
# !python -m pip install --upgrade pymupdf
import fitz
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"


In [2]:
from functions import extract_all_text2
import embedding_functions

/home/gsilvi/miniforge3/envs/PaperOracle/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
all_tex = extract_all_text2('papers/2101.00689/Npi_paper.tex')

In [4]:
final_text = embedding_functions.texStripper2(all_tex,'title','abstract') # refine the text

KEYS: dict_keys([])
SECTIONS: dict_keys(['-', 'author', 'affiliation', 'title', 'date', 'abstract', 'Introduction', 'Gauge Ensemble', 'Interpolating Operators', 'Wick contractions', 'Spectra results', 'L{\\"u}scher quantization conditions}\n', 'Results for the scattering amplitudes', 'Parametrizations used', 'Fit procedure and results', 'Conclusions', 'Acknowledgments', 'One-to-one mapping of energy levels to phase shifts in irreps without mixing between $J=1/2$ and $J=3/2$', 'Transformation properties of operators', '\\texorpdfstring{Matrices $\\mathcalM'])
sec='-'
lines=[57, 62, 65, 69, 70, 74, 78, 81, 84, 87, 90, 93, 94, 98, 103, 106, 116, 118, 119, 120]
sec='author'
lines=[58, 63, 66, 71, 75, 79, 82, 85, 88, 91, 95, 99]
sec='affiliation'
lines=[60, 61, 64, 67, 68, 72, 73, 76, 77, 80, 83, 86, 89, 92, 96, 97, 100, 101, 102]
sec='title'
lines=[104]
sec='date'
lines=[105]
sec='abstract'
lines=[108]
sec='Introduction'
lines=[123]
sec='Gauge Ensemble'
lines=[150]
KEYWORD: \end{table} 153

Token indices sequence length is longer than the specified maximum sequence length for this model (1698 > 1024). Running this sequence through the model will result in indexing errors


Phrase too long: 3368  at lines: [203, 205, 210, 212, 214, 216, 219, 221]
Splitting phrase in  1  parts
Phrase too long: 5290  at lines: [285, 286, 288, 289, 291, 292, 293, 294, 295, 297, 298, 299, 301, 302, 303, 304, 306, 307, 308, 309, 310, 312, 313, 314, 316, 317, 318]
Splitting phrase in  1  parts
Phrase too long: 3647  at lines: [456, 458, 459, 461, 462, 464, 465, 466, 468, 469, 471, 472, 473, 474, 475, 477, 478, 479, 481, 482, 484, 485]
Splitting phrase in  1  parts
Phrase too long: 2365  at lines: [601, 602, 608, 609, 614, 618, 621]
Splitting phrase in  1  parts
Phrase too long: 2089  at lines: [627, 631, 635, 637, 641]
Splitting phrase in  1  parts
Phrase too long: 4059  at lines: [722, 727, 730, 733, 736, 739, 742]
Splitting phrase in  1  parts
Phrase too long: 2041  at lines: [798, 799, 800, 801, 804, 806]
Splitting phrase in  1  parts
Phrase too long: 2446  at lines: [925, 927, 928, 930, 931, 933, 934, 936, 937, 940, 942, 943]
Splitting phrase in  1  parts
Phrase too long: 8

In [5]:
import pandas as pd
for file in final_text.keys():
    df = pd.DataFrame(final_text[file]['full'], columns=['Phrase','Lines'])

# fix the fact that \begin{} and end{} should not be asked in SyncTex, Which means when lines start with 
# \begin{ or \end{, they should not be added to the list of lines to be highlighted


In [6]:
import subprocess
# papers/2101.00689/Npi_paper.tex
def call_syncTex(dataframe, path_to_pdf, path_to_tex):
    # add a colum Page to the dataframe
    dataframe['Page'] = None
    
    gen_dict = {}
    lines_in_Tex = []
    for i in range(len(df)):
        collection = []
        lines_in_Tex = df.loc[i,'Lines']
        
        
        for row in lines_in_Tex:
            # print(f"{row=}")
            result = subprocess.run(['synctex', 
                                    'view',
                                    '-i',f'{row}:0:{path_to_tex}',
                                    '-o',f'{path_to_pdf}' ], 
                                    stdout=subprocess.PIPE).stdout.decode('utf-8')
            # print(result)
            
            # print(f"{result=}")
            
            for item in result.split('\n'):
                if ':' in item:
                    key, value = item.split(':')
                    gen_dict[key] = value
                    gen_dict['TeX_line'] = row
                    if key=='after':
                        # if 228<=gen_dict['TeX_line']<=230:
                        collection.append(gen_dict)
                        gen_dict = {}

            
            # print(f"{collection=} {row=}")
        text_instances = set()
        for stuck in collection:
            if float(stuck['W'])>0:
                if float(stuck['y'])<float(stuck['v']):
                    text_instances.add((stuck['Page'],
                                        float(stuck['h']),
                                        # float(stuck['v'])-float(stuck['H']),
                                        float(stuck['y'])-9,
                                        # max(float(stuck['y'])-9,float(stuck['v'])-float(stuck['H'])),
                                        float(stuck['h'])+float(stuck['W']),
                                        float(stuck['v']))
                                        )
                else:
                    text_instances.add((stuck['Page'],
                                        float(stuck['h']),
                                        float(stuck['v'])-9,
                                        float(stuck['h'])+float(stuck['W']),
                                        float(stuck['v']))
                                )
        for ii, inst in enumerate(text_instances):
            # divide the tuple inst into its components
            # print(f"{inst=}")
            (pageNum, x0, y0, x1, y1) = inst 
            # if it already exists, append pageNum and the 4 coordinates as a list to the dataframe
            if df.loc[i,'Page'] != None:
                df.loc[i,'Page'].append(pageNum)
                df.loc[i,'Highlight'].append(str([x0, y0, x1, y1]))
            # if it does not exist, create a new row in the dataframe
            else:
                df.loc[i,'Page'] = [pageNum]
                df.loc[i,'Highlight'] = [str([x0, y0, x1, y1])]
    return df

                




In [7]:
df = call_syncTex(df, 'papers/2101.00689/Npi_paper.pdf', 'papers/2101.00689/Npi_paper.tex')

In [ ]:
# loop over the dataframe and highlight the text
doc = fitz.open('papers/2101.00689/Npi_paper.pdf')
for i in range(len(df)):
    # skip the NaN values
    if type(df.loc[i,'Page'])==list:
        for j in range(len(df.loc[i,'Page'])):
            page = doc[int(df.loc[i,'Page'][j])-1]
            rectangle = fitz.Rect(eval(df.loc[i,'Highlight'][j]))
            highlight = page.add_highlight_annot(rectangle)
            highlight.update()

doc.save('papers/2101.00689/Npi_paper_highlighted.pdf', garbage=4, deflate=True, clean=True)


#STOP

In [ ]:
#
doc = fitz.open("papers/2101.00689/Npi_paper.pdf")
for page in doc:
    ### SEARCH
    print(page.number)
    
    for ii, inst in enumerate(df):
        
        # divide the tuple inst into its components
        (pageNum, x0, y0, x1, y1) = inst 
        if pageNum==str(page.number+1):
            highlight = page.add_highlight_annot(fitz.Rect(x0, y0, x1, y1))
            highlight.update()
    # for inst in text_instances2:
   
    #     # divide the tuple inst into its components
    #     (pageNum, x0, y0, x1, y1) = inst 
    #     if pageNum==str(page.number+1):
    #         print(inst)
    #         highlight = page.add_underline_annot(fitz.Rect(x0, y0, x1, y1))
    #         highlight.update()
### OUTPUT

doc.save("output.pdf", garbage=4, deflate=True, clean=True)

In [18]:
#set the environment variable TOKENIZERS_PARALLELISM=true
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# extract line 36 from df
import subprocess
# line_in_df = 15
# get all Lines from df
lines = []
for i in range(len(df)):
    lines.extend(df.loc[i,'Lines'])
# print(lines)

result = ''
collection = list()
gen_dict = {}

line_in_df = 11
lines = df.iloc[line_in_df].Lines
print(lines)
for row in lines:
    # print('Row',row)
    result = subprocess.run(['synctex', 'view', '-i',f'{row}:0:papers/2101.00689/Npi_paper.tex','-o','papers/2101.00689/Npi_paper.pdf' ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    for stuck in result.split('\n'):
        
        if ':' in stuck:
            key, value = stuck.split(':')
            gen_dict[key] = value
            gen_dict['TeX_line'] = row
            if key=='after':
                # if 228<=gen_dict['TeX_line']<=230:
                collection.append(gen_dict)
                gen_dict = {}
    
import pprint
# pprint.pprint(collection)

# height from y to v
# width from x to h+W

[131, 132]
This is SyncTeX command line utility, version 1.5
SyncTeX result begin
Output:papers/2101.00689/Npi_paper.pdf
Page:1
x:320.376617
y:662.376038
h:314.966400
v:664.866699
W:225.033615
H:9.962640
before:
offset:-1
middle:
after:
Output:papers/2101.00689/Npi_paper.pdf
Page:1
x:321.663513
y:616.547913
h:314.966400
v:619.038574
W:225.033615
H:9.962640
before:
offset:-1
middle:
after:
Output:papers/2101.00689/Npi_paper.pdf
Page:1
x:320.501190
y:673.833069
h:314.966400
v:676.323730
W:225.033615
H:9.962640
before:
offset:-1
middle:
after:
Output:papers/2101.00689/Npi_paper.pdf
Page:1
x:337.382355
y:605.090881
h:314.966400
v:607.028076
W:225.033615
H:8.855677
before:
offset:-1
middle:
after:
Output:papers/2101.00689/Npi_paper.pdf
Page:1
x:340.454193
y:650.919006
h:314.966400
v:653.409668
W:225.033615
H:9.962640
before:
offset:-1
middle:
after:
Output:papers/2101.00689/Npi_paper.pdf
Page:1
x:332.124298
y:628.004944
h:314.966400
v:629.942139
W:225.033615
H:8.855677
before:
offset:-1
mid

In [ ]:
# h v represent the beginning of the line x and y
# x0<x1 , y0<y1, increase number means go down or right

#0._______
# |       | 
# |_______.1


### READ IN PDF
doc = fitz.open("papers/2101.00689/Npi_paper.pdf")
# text_instances = [fitz.Rect(132,489, 296,500)]
text_instances = set()
text_instances2 = set()

for stuck in collection:
    #yellow
    if float(stuck['W'])>0:
        if float(stuck['y'])<float(stuck['v']):
            text_instances.add((stuck['Page'],
                                float(stuck['h']),
                                # float(stuck['v'])-float(stuck['H']),
                                float(stuck['y'])-9,
                                # max(float(stuck['y'])-9,float(stuck['v'])-float(stuck['H'])),
                                float(stuck['h'])+float(stuck['W']),
                                float(stuck['v']))
                                )
        else:
            text_instances.add((stuck['Page'],
                                float(stuck['h']),
                                float(stuck['v'])-9,
                                float(stuck['h'])+float(stuck['W']),
                                float(stuck['v']))
                            )
    # #green
    # #if float(stuck['y'])<float(stuck['v']):
    # text_instances.add((stuck['Page'],
    #                         float(stuck['x'])-10,
    #                         float(stuck['y'])-10,
    #                         float(stuck['x']),
    #                         float(stuck['y']),
    #                         )
    #                         )
    # text_instances.add((stuck['Page'],
    #                         float(stuck['h']),
    #                         float(stuck['v'])-float(stuck['H']),
    #                         float(stuck['x']),
    #                         float(stuck['v']),
    #                         )
    #                         )
    
    ### HIGHLIGHT
for page in doc:
    ### SEARCH
    print(page.number)
    
    for ii, inst in enumerate(text_instances):
        
        # divide the tuple inst into its components
        (pageNum, x0, y0, x1, y1) = inst 
        if pageNum==str(page.number+1):
            highlight = page.add_highlight_annot(fitz.Rect(x0, y0, x1, y1))
            highlight.update()
    # for inst in text_instances2:
   
    #     # divide the tuple inst into its components
    #     (pageNum, x0, y0, x1, y1) = inst 
    #     if pageNum==str(page.number+1):
    #         print(inst)
    #         highlight = page.add_underline_annot(fitz.Rect(x0, y0, x1, y1))
    #         highlight.update()
### OUTPUT

doc.save("output.pdf", garbage=4, deflate=True, clean=True)
